# HW#2 Kaggle Challenge

In this homework, you will train a model to predict back order.

However, the dataset is highly imbalanced.

You will need to deal with this problem and train a model to achieve the best performance.

In [44]:
import pandas as pd
import numpy as np

## Load Dataset

We will use the Pandas library for loading `csv` files into `DataFrame`, which is a 2-dimension table.

In [45]:
import pandas as pd

In [46]:
train_data = pd.read_csv("./train_data.csv")
train_data.head()

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,3520988,8.0,8.0,0.0,0.0,0.0,0.0,0.0,3.0,11.0,...,0.0,0.99,0.99,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1357206,18.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.86,0.84,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1857911,1089.0,8.0,0.0,0.0,0.0,0.0,1.0,3.0,5.0,...,0.0,0.50,0.66,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,3455231,70.0,8.0,2.0,42.0,88.0,123.0,11.0,45.0,95.0,...,1.0,0.70,0.71,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,1316436,57.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.96,0.96,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [47]:
test_data = pd.read_csv("./test_data.csv")
test_data.head()

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop
0,3384866,18.0,9.0,10.0,0.0,0.0,0.0,33.0,39.0,53.0,...,0.0,0.0,0.94,0.83,0.0,0.0,0.0,0.0,1.0,0.0
1,3029666,770.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.07,0.21,0.0,0.0,0.0,0.0,1.0,0.0
2,3188469,881.0,9.0,0.0,0.0,0.0,0.0,5.0,19.0,39.0,...,0.0,0.0,0.85,0.82,4.0,0.0,0.0,0.0,1.0,0.0
3,2149770,1.0,8.0,0.0,8.0,8.0,8.0,1.0,2.0,5.0,...,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,1.0,0.0
4,1853225,8.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.62,0.77,0.0,1.0,0.0,0.0,1.0,0.0


## Train a simple Logistic Regression model

I will use the `LogisticRegression` model from `sklearn` library.

For more information about the model, please refer to the documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

from imblearn.over_sampling import SMOTE

In [ ]:
y = train_data["went_on_backorder"]
X_train = train_data.drop(columns=["sku", "went_on_backorder"])

binary_cols = ["potential_issue", "deck_risk", "oe_constraint", "ppap_risk", "stop_auto_buy", "rev_stop"]

X_train["inv_to_sales_1m"] = X_train["national_inv"] / (X_train["sales_1_month"] + 1)
X_train["inv_to_forecast_3m"] = X_train["national_inv"] / (X_train["forecast_3_month"] + 1)
X_train["sales_ratio_1_3"] = X_train["sales_1_month"] / (X_train["sales_3_month"] + 1)
X_train["sales_ratio_3_6"] = X_train["sales_3_month"] / (X_train["sales_6_month"] + 1)
X_train["sales_ratio_1_9"] = X_train["sales_1_month"] / (X_train["sales_9_month"] + 1)
X_train["adjusted_lead_time"] = X_train["lead_time"] / (X_train["perf_6_month_avg"] + 0.01)
X_train["log_sales_1m"] = np.log1p(X_train["sales_1_month"])
X_train["log_inv"] = np.log1p(X_train["national_inv"])
X_train["log_past_due"] = np.log1p(X_train["pieces_past_due"])
X_train["forecast_ratio"] = X_train["forecast_3_month"] / (X_train["forecast_9_month"] + 1)
X_train["avg_perf"] = (X_train["perf_6_month_avg"] + X_train["perf_12_month_avg"]) / 2
X_train["delta_perf"] = X_train["perf_6_month_avg"] - X_train["perf_12_month_avg"]
X_train["any_risk"] = X_train[binary_cols].max(axis=1)
X_train["total_risk_flags"] = X_train[binary_cols].sum(axis=1)
X_train["zero_sales_1m"] = (X_train["sales_1_month"] == 0).astype(int)
X_train["no_forecast"] = (X_train["forecast_3_month"] == 0).astype(int)
X_train["forecast_to_sales"] = X_train["forecast_3_month"] / (X_train["sales_3_month"] + 1)


X_train.head()

/Users/seoyeongkim/Desktop/SKKU/2025-1/기계학습원론/SWE3050-Fundamentals-of-Machine-Learning/HW2/venv/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/seoyeongkim/Desktop/SKKU/2025-1/기계학습원론/SWE3050-Fundamentals-of-Machine-Learning/HW2/venv/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,...,log_inv,log_past_due,forecast_ratio,avg_perf,delta_perf,any_risk,total_risk_flags,zero_sales_1m,no_forecast,forecast_to_sales
0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,3.0,11.0,13.0,...,2.197225,0.000000,0.00000,0.990,0.00,1.0,2.0,1,1,0.000000
1,18.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.944439,0.000000,0.00000,0.850,0.02,1.0,1.0,1,1,0.000000
2,1089.0,8.0,0.0,0.0,0.0,0.0,1.0,3.0,5.0,9.0,...,6.993933,0.000000,0.00000,0.580,-0.16,1.0,2.0,0,1,0.000000
3,70.0,8.0,2.0,42.0,88.0,123.0,11.0,45.0,95.0,157.0,...,4.262680,0.693147,0.33871,0.705,-0.01,1.0,2.0,0,0,0.913043
4,57.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.060443,0.000000,0.00000,0.960,0.00,1.0,1.0,1,1,0.000000


In [49]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)

In [50]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [51]:
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y)

In [52]:
clf = LogisticRegression(penalty="l1", class_weight="balanced", max_iter=5000, solver="liblinear")
clf.fit(X_train_res, y_train_res)

LogisticRegression(class_weight='balanced', max_iter=5000, penalty='l1',
                   solver='liblinear')

In [53]:
probs_val = clf.predict_proba(X_train_res)[:, 1]
preds_val = (probs_val >= 0.5).astype(int)

In [54]:
print("F1 score: ", f1_score(y_train_res, preds_val))
print("ROC AUC: ", roc_auc_score(y_train_res, probs_val))
print("Accuracy score: ", accuracy_score(y_train_res, preds_val))
print(classification_report(y_train_res, preds_val))

F1 score:  0.8696639641852975
ROC AUC:  0.9303801272222223
Accuracy score:  0.8673722222222222
              precision    recall  f1-score   support

         0.0       0.88      0.85      0.86     90000
         1.0       0.85      0.88      0.87     90000

    accuracy                           0.87    180000
   macro avg       0.87      0.87      0.87    180000
weighted avg       0.87      0.87      0.87    180000



In [ ]:
test_X = test_data.drop(columns=["sku"])

test_X["inv_to_sales_1m"] = test_X["national_inv"] / (test_X["sales_1_month"] + 1)
test_X["inv_to_forecast_3m"] = test_X["national_inv"] / (test_X["forecast_3_month"] + 1)
test_X["sales_ratio_1_3"] = test_X["sales_1_month"] / (test_X["sales_3_month"] + 1)
test_X["sales_ratio_3_6"] = test_X["sales_3_month"] / (test_X["sales_6_month"] + 1)
test_X["sales_ratio_1_9"] = test_X["sales_1_month"] / (test_X["sales_9_month"] + 1)
test_X["adjusted_lead_time"] = test_X["lead_time"] / (test_X["perf_6_month_avg"] + 0.01)
test_X["log_sales_1m"] = np.log1p(test_X["sales_1_month"])
test_X["log_inv"] = np.log1p(test_X["national_inv"])
test_X["log_past_due"] = np.log1p(test_X["pieces_past_due"])
test_X["forecast_ratio"] = test_X["forecast_3_month"] / (test_X["forecast_9_month"] + 1)
test_X["avg_perf"] = (test_X["perf_6_month_avg"] + test_X["perf_12_month_avg"]) / 2
test_X["delta_perf"] = test_X["perf_6_month_avg"] - test_X["perf_12_month_avg"]
test_X["any_risk"] = test_X[binary_cols].max(axis=1)
test_X["total_risk_flags"] = test_X[binary_cols].sum(axis=1)
test_X["zero_sales_1m"] = (test_X["sales_1_month"] == 0).astype(int)
test_X["no_forecast"] = (test_X["forecast_3_month"] == 0).astype(int)
test_X["forecast_to_sales"] = test_X["forecast_3_month"] / (test_X["sales_3_month"] + 1)

/Users/seoyeongkim/Desktop/SKKU/2025-1/기계학습원론/SWE3050-Fundamentals-of-Machine-Learning/HW2/venv/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/seoyeongkim/Desktop/SKKU/2025-1/기계학습원론/SWE3050-Fundamentals-of-Machine-Learning/HW2/venv/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [56]:
test_X.replace([np.inf, -np.inf], np.nan, inplace=True)
test_X.fillna(0, inplace=True)

In [57]:
test_X_scaled = scaler.transform(test_X)

In [58]:
probs_test = clf.predict_proba(test_X_scaled)[:, 1]
pred_test = (probs_test >= 0.5).astype(int)

### Make predictions on the test data

The evaluation is done on Kaggle, so we need to make predictions on the test data and submit the predictions to Kaggle.

The file format should be as follows:
```
Index,Class
0,0.0
1,0.0
2,1.0
...
```

In [59]:
submission = pd.DataFrame({"ID": range(len(test_data)), "Class": pred_test})
submission.head(5)

,ID,Class
0,0,1
1,1,0
2,2,0
3,3,1
4,4,0


In [60]:
submission.to_csv("submission_last.csv", index=False)

## Some tips

You can also use the LogisticRegression Code implemented directly in problem 1.

There are many ways to improve the model performance (e.g. feature engineering, hyperparameter tuning, etc.). You can try different methods to improve your model performance.

Also, you could try various methods to deal with the imbalanced dataset problem (e.g. oversampling, undersampling, class weight, etc.).

If you have any questions regarding the homework, feel free to ask on the anonymous bulletin board.

Good luck!